In [1]:
# !pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# Only if you have a GPU

In [2]:



from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices()) 
tf.config.list_physical_devices('GPU')

['/device:CPU:0']


[]

In [3]:
######## GPU CONFIGS FOR RTX 2070 ###############
## Please ignore if not training on GPU       ##
## this is important for running CuDNN on GPU ##

# check if GPU can be seen by TF
tf.config.list_physical_devices('GPU')
# enable this line if you wish to see whether GPU/CPU executes
# each command
#tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
###############################################

# Load Data

In [4]:
# see available tfds data sets
", ".join(tfds.list_builders())

'abstract_reasoning, accentdb, aeslc, aflw2k3d, ag_news_subset, ai2_arc, ai2_arc_with_ir, amazon_us_reviews, anli, arc, bair_robot_pushing_small, bccd, beans, big_patent, bigearthnet, billsum, binarized_mnist, binary_alpha_digits, blimp, bool_q, c4, caltech101, caltech_birds2010, caltech_birds2011, cars196, cassava, cats_vs_dogs, celeb_a, celeb_a_hq, cfq, cherry_blossoms, chexpert, cifar10, cifar100, cifar10_1, cifar10_corrupted, citrus_leaves, cityscapes, civil_comments, clevr, clic, clinc_oos, cmaterdb, cnn_dailymail, coco, coco_captions, coil100, colorectal_histology, colorectal_histology_large, common_voice, coqa, cos_e, cosmos_qa, covid19, covid19sum, crema_d, curated_breast_imaging_ddsm, cycle_gan, d4rl_adroit_door, d4rl_adroit_hammer, d4rl_adroit_pen, d4rl_adroit_relocate, d4rl_mujoco_ant, d4rl_mujoco_halfcheetah, d4rl_mujoco_hopper, d4rl_mujoco_walker2d, dart, davis, deep_weeds, definite_pronoun_resolution, dementiabank, diabetic_retinopathy_detection, div2k, dmlab, doc_nli, do

In [5]:
# using TFDS dataset
# note: as_supervised converts dicts to tuples
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train", 
                                with_info=True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews", split="test", 
                      as_supervised=True)

In [6]:
print(ds_info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='C:\\Users\\lukem\\tensorflow_datasets\\imdb_reviews\\plain_text\\1.0.0',
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        'train':

In [7]:
for example, label in imdb_train.take(1):
    print(example, '\n', label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


In [8]:
# Use the default tokenizer settings
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
MAX_TOKENS = 0

for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)

In [9]:
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, 
                                                   tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size

print(vocab_size, MAX_TOKENS)

93931 2525


In [10]:
# Lets verify tokenization and encoding works
for example, label in imdb_train.take(1):
    print(example)
    encoded = imdb_encoder.encode(example.numpy())
    print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This 

In [11]:
# transformation fucntions to be used with the dataset
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
    encoded = imdb_encoder.encode(sample.numpy())
    pad = sequence.pad_sequences([encoded], padding='post', 
                                 maxlen=150)
    return np.array(pad[0], dtype=np.int64)  


def encode_tf_fn(sample, label):
    encoded = tf.py_function(encode_pad_transform, 
                                       inp=[sample], 
                                       Tout=(tf.int64))
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label

In [12]:
# test the transformation on a small subset
subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)

In [13]:
for review, label in tst.take(1):
    print(review, label)
    print(imdb_encoder.decode(review))

tf.Tensor(
[88995 69667 88990  1707 25264 53864 30424 38705 43359 29772 91140 25947
 70621 17761 26734 78285 44754 92453 62305 30438 18154 61228 69805  7446
 68264 43359 75752 43540  6642 91140 31806 82836 75752 30438 37908  4941
 84805 77578 69805 53864 43166 54698 82791 88995 53864   995 88990 71927
 46216 16065  2830 88364 67858 27828 69001 67242 81867 37041 39488  5183
 35505 31013 81867 26803 75752 32728 55124 45418  9683 21095 34848 43968
 50487 90516 39766 48623 82884 23831 35484 17761 69667 46082 61228 50950
 69001 50522   420 91140 50950 53864 92633 69667 61114 68214 68832 82485
 48694  4819 76842 56931 92633 43254 62305 54963 66876 69805 28104 18715
 43166 66876 70621 17761 43166 71226 44483  4819  4941 66703 78838 66148
 54595     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

In [14]:
#now tokenize/encode/pad all training
# and testing data
encoded_train = imdb_train.map(encode_tf_fn,
                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn,
                             num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Preparing the model

In [15]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=100

In [16]:
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units),
    # introduce a dropout layer after the LSTM layer
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model


In [17]:
model = build_model_lstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (100, None, 64)           6011584   
                                                                 
 lstm (LSTM)                 (100, 64)                 33024     
                                                                 
 dense (Dense)               (100, 1)                  65        
                                                                 
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

# Model Training and Evaluation

In [19]:
# Prefetch for performance
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [20]:
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 88s 336ms/step - loss: 0.4083 - accuracy: 0.8051 - precision: 0.7877 - recall: 0.8354
Epoch 2/10
250/250 [==============================] - 82s 327ms/step - loss: 0.1685 - accuracy: 0.9396 - precision: 0.9404 - recall: 0.9386
Epoch 3/10
250/250 [==============================] - 84s 337ms/step - loss: 0.1038 - accuracy: 0.9647 - precision: 0.9655 - recall: 0.9638
Epoch 4/10
250/250 [==============================] - 87s 348ms/step - loss: 0.0814 - accuracy: 0.9722 - precision: 0.9726 - recall: 0.9717
Epoch 5/10
250/250 [==============================] - 83s 330ms/step - loss: 0.0546 - accuracy: 0.9815 - precision: 0.9828 - recall: 0.9802
Epoch 6/10
250/250 [==============================] - 63s 252ms/step - loss: 0.0373 - accuracy: 0.9884 - precision: 0.9897 - recall: 0.9871
Epoch 7/10
250/250 [==============================] - 83s 331ms/step - loss: 0.0477 - accuracy: 0.9834 - precision: 0.9833 - recall: 0.9834
Epoch 8/10
250/250 [

In [21]:
model.evaluate(encoded_test.batch(BATCH_SIZE))

250/250 [==============================] - 27s 100ms/step - loss: 0.7815 - accuracy: 0.8352 - precision: 0.8187 - recall: 0.861017s - loss: 0.8059 - accuracy: 0.8326 - precision: 0.8134 -  - ETA: 16s


[0.7814833521842957,
 0.8352000117301941,
 0.8187281489372253,
 0.8610399961471558]

# BiLSTM Model

In [22]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=50

In [23]:
dropout=0.2
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [24]:
bilstm = build_model_bilstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

bilstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (50, None, 128)           12023168  
                                                                 
 dropout (Dropout)           (50, None, 128)           0         
                                                                 
 bidirectional (Bidirectiona  (50, None, 128)          98816     
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (50, None, 128)           0         
                                                                 
 bidirectional_1 (Bidirectio  (50, 128)                98816     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (50, 128)                

In [25]:
bilstm.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

In [26]:
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [27]:
bilstm.fit(encoded_train_batched, epochs=5)

Epoch 1/5
500/500 [==============================] - 419s 809ms/step - loss: 0.3729 - accuracy: 0.8278 - precision: 0.8245 - recall: 0.8329
Epoch 2/5
500/500 [==============================] - 275s 550ms/step - loss: 0.1532 - accuracy: 0.9456 - precision: 0.9461 - recall: 0.9450
Epoch 3/5
500/500 [==============================] - 283s 566ms/step - loss: 0.0742 - accuracy: 0.9750 - precision: 0.9757 - recall: 0.9742
Epoch 4/5
500/500 [==============================] - 276s 552ms/step - loss: 0.0523 - accuracy: 0.9819 - precision: 0.9823 - recall: 0.9815
Epoch 5/5
500/500 [==============================] - 275s 550ms/step - loss: 0.0384 - accuracy: 0.9869 - precision: 0.9876 - recall: 0.9862


In [28]:
bilstm.evaluate(encoded_test.batch(BATCH_SIZE))

500/500 [==============================] - 17s 35ms/step - loss: 0.6753 - accuracy: 0.8384 - precision: 0.8459 - recall: 0.8274


[0.6753188967704773,
 0.8383600115776062,
 0.8459147810935974,
 0.8274400234222412]